In [9]:
from huggingface_hub.hf_api import HfFolder 
from probert import ProBertConfig, ProBERT
from transformers import AutoConfig, AutoModel
from pytorch_pretrained_bert.modeling import WEIGHTS_NAME, CONFIG_NAME
import os
import torch

from dotenv import load_dotenv
load_dotenv()

True

In [10]:
# private hf model token needed. give write persmission if push to hub is expected
# save in .env file with HF_TOKEN key
# see https://huggingface.co/docs/transformers.js/guides/private
HfFolder.save_token(os.environ.get("HF_TOKEN"))

In [11]:
ProBertConfig.register_for_auto_class()
ProBERT.register_for_auto_class("AutoModel")

In [12]:
model_path = os.path.abspath("../../results/probert/ensembled_lms/bert_large_uncased/ensembled_seeds/42/train_evaluate_cv/1/probert/1/model")
output_model_file = os.path.join(model_path, WEIGHTS_NAME)
output_config_file = os.path.join(model_path, CONFIG_NAME)
probert_config = ProBertConfig()
probert_model = ProBERT(probert_config)

if torch.cuda.is_available():
    map_location=lambda storage, loc: storage.cuda()
else:
    map_location='cpu'

device = torch.device("cuda" if torch.cuda.is_available() and not False else "cpu")

print('Loading Model.')
probert_model.load_state_dict(torch.load(output_model_file, map_location=map_location), strict=False)
probert_model.to(device)

Loading Model.


ProBERT(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
      

In [13]:
# Register custom class
AutoConfig.register("probert", ProBertConfig)
AutoModel.register(ProBertConfig, ProBERT)

In [17]:
probert_model.push_to_hub("ibm/probert", revision="probert-singular-v1", token=True)

CommitInfo(commit_url='https://huggingface.co/ibm/probert/commit/986fb28014f9debe3b066bf2835581937b461f55', commit_message='Upload ProBERT', commit_description='', oid='986fb28014f9debe3b066bf2835581937b461f55', pr_url=None, pr_revision=None, pr_num=None)